In [88]:
import pandas as pd
import os
import numpy as np
from PIL import Image

# 파일 경로 설정
csv_file_path = "C:/Users/amyko/Desktop/G1020/G1020.csv"
image_dir = "C:/Users/amyko/Desktop/G1020/Images_Cropped/img"

# CSV 파일 읽기
data = pd.read_csv(csv_file_path)

# 이미지 파일 리스트 가져오기
image_files = os.listdir(image_dir)
image_files = [f for f in image_files if f.endswith('.jpg')]

images = []
labels = []

# 이미지와 레이블을 가져오기
for index, row in data.iterrows():
    img_path = os.path.join(image_dir, row['imageID'])
    img = Image.open(img_path)
    img = img.resize((480, 480))
    img = np.array(img) / 255.0  # 정규화
    images.append(img)  # images 리스트에 모든 이미지 저장
    labels.append(row['binaryLabels'])  # labels 리스트에 모든 이미지의 정답값 저장

# 데이터 분할
split_index = int(len(images) * 0.8)
x_train, x_test = np.array(images[:split_index]).reshape(-1, 480 * 480 * 3), np.array(images[split_index:]).reshape(-1, 480 * 480 * 3)
t_train, t_test = np.array(labels[:split_index]), np.array(labels[split_index:])

print('Train set size:', len(x_train))
print('Test set size :', len(x_test))

# 신경망 클래스 정의
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        # 가중치 초기화
        self.weights1 = np.random.randn(input_size, hidden_size) * 0.01
        self.weights2 = np.random.randn(hidden_size, output_size) * 0.01
        self.learning_rate = learning_rate

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, x):
        self.hidden = self.sigmoid(np.dot(x, self.weights1))
        self.output = self.sigmoid(np.dot(self.hidden, self.weights2))
        return self.output

    def backward(self, x, y):
        # 오차 계산
        output_error = y - self.output
        output_delta = output_error * self.sigmoid_derivative(self.output)

        hidden_error = np.dot(output_delta, self.weights2.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden)

        # 가중치 업데이트
        self.weights2 += np.dot(self.hidden.T, output_delta) * self.learning_rate
        self.weights1 += np.dot(x.T, hidden_delta) * self.learning_rate

    def accuracy(self, x, y):
        predictions = self.forward(x)
        predicted_classes = (predictions > 0.5).astype(int).flatten()
        return np.mean(predicted_classes == y)

    def train(self, x, y, epochs=500, iter_per_epoch=100):
        train_acc_list = []
        test_acc_list = []

        for i in range(epochs):
            self.forward(x)
            self.backward(x, y)

            # 특정 에폭마다 정확도 계산
            if i % iter_per_epoch == 0:
                train_acc = self.accuracy(x_train, t_train)
                test_acc = self.accuracy(x_test, t_test)
                train_acc_list.append(train_acc)
                test_acc_list.append(test_acc)
                print(f'Epoch {i}: Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
        
        return train_acc_list, test_acc_list

# 신경망 인스턴스 생성
input_size = 480 * 480 * 3  # 입력 크기
hidden_size = 128  # 은닉층 크기
output_size = 1  # 출력 크기 (이진 분류)
nn = SimpleNeuralNetwork(input_size, hidden_size, output_size)

# 레이블을 1D 배열로 변환
t_train = t_train.reshape(-1, 1)

# 모델 훈련
train_acc_list, test_acc_list = nn.train(x_train, t_train, epochs=500, iter_per_epoch=100)

# 최종 평가
final_test_acc = nn.accuracy(x_test, t_test)
print(f'Final Test accuracy: {final_test_acc:.4f}')


Train set size: 816
Test set size : 204
Epoch 0: Train Acc: 0.7083, Test Acc: 0.7157
Epoch 100: Train Acc: 0.7083, Test Acc: 0.7157
Epoch 200: Train Acc: 0.7083, Test Acc: 0.7157
Epoch 300: Train Acc: 0.7083, Test Acc: 0.7157
Epoch 400: Train Acc: 0.7083, Test Acc: 0.7157
Final Test accuracy: 0.7157
